<p style="font-size:25px; color:#496595"> ¿Qué se hará? </p>

><b>En esta sección se llevan a cabo dos pasos clave:</b>
>- <b>Extracción de datos alojados en Google Sheets:</b>
En esta etapa, se accede y se extraen los datos almacenados en una hoja de cálculo de Google Sheets. Esto implica establecer la conexión con la hoja de cálculo utilizando las credenciales adecuadas y la API de Google Sheets, y luego recuperar los datos necesarios para su posterior análisis.

>- <b>Entendimiento o exploración de los datos:</b>
En esta fase, se realiza un análisis inicial de los datos extraídos para obtener una comprensión más profunda de su estructura, contenido y calidad. Se exploran las diferentes variables presentes en los datos, se identifican valores atípicos o faltantes. Este proceso ayuda a descubrir posibles problemas en los datos, lo que orientará las decisiones posteriores de limpieza, transformación y análisis.</b>



<p style="font-size:25px; color:#496595">Sobre el conjunto de datos</p>

> Este conjunto de datos pertenece a una tienda global estadounidense que ofrece sus productos en más de 520 ciudades y 49 estados. La tienda se especializa en la venta de una amplia variedad de productos, incluyendo muebles, suministros de oficina y tecnología, como teléfonos y electrodomésticos.

>Los datos recopilados abarcan un período de tres años, desde 2015 hasta 2018. Algunos de los campos incluidos en el conjunto de datos son: fechas de pedido y entrega, modo de envío, nombres de clientes, país, ciudad, región, nombre del producto, categoría, subcategoría, segmento y ventas.

<p style="font-size:25px; color:#496595">1. Extracción</p>


In [14]:
#%pip install gspread-pandas oauth2client #Instalación de libreria para obtener datos de google sheets

#==== Importación de librerias a usar ====
import pandas as pd
import gspread_pandas as gpd #biblioteca que simplifica la integración entre Pandas y Google Sheets para extracción de datos
import re

In [6]:
#==== Extrayendo datos de google sheet ====
#Conexión a los datos alojados en google sheets, mas info de como hacer conexión en https://www.youtube.com/watch?v=jeZWv5PQJAk
gc = gpd.Spread('https://docs.google.com/spreadsheets/d/1ZZLYXQNGYOsFoiW8viTRKy-WCF8TPfpFF6UlHrMMvDw/edit?usp=sharing')
gc.open_sheet('Train')
df = gc.sheet_to_df() #Dataframe contenedor de los datos
df=df.reset_index()

<p style="font-size:25px; color:#496595">2. Exploración de datos</p>



In [7]:
#=== Visualización primeras 5 filas del dataframe===
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,9.575.775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368


In [8]:
#===Información sobre el dataframe ===
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Row ID         9800 non-null   object
 1   Order ID       9800 non-null   object
 2   Order Date     9800 non-null   object
 3   Ship Date      9800 non-null   object
 4   Ship Mode      9800 non-null   object
 5   Customer ID    9800 non-null   object
 6   Customer Name  9800 non-null   object
 7   Segment        9800 non-null   object
 8   Country        9800 non-null   object
 9   City           9800 non-null   object
 10  State          9800 non-null   object
 11  Postal Code    9800 non-null   object
 12  Region         9800 non-null   object
 13  Product ID     9800 non-null   object
 14  Category       9800 non-null   object
 15  Sub-Category   9800 non-null   object
 16  Product Name   9800 non-null   object
 17  Sales          9800 non-null   object
dtypes: object(18)
memory usage: 

In [9]:
#===Numero de filas y columnas ===
df.shape #9800 filas, 17 columnas

(9800, 18)

In [10]:
#===Explorando datos categoricos y registros unicos ===
df["Ship Mode"].unique() #'Second Class', 'Standard Class', 'First Class', 'Same Day'2
df.Segment.unique() #'Consumer', 'Corporate', 'Home Office'
df.Country.unique() #'United States'
len(df.City.unique()) #529 registros
df.State.unique() #49 registros
df.Region.unique() #'South', 'West', 'Central', 'East'
df.Category.unique() #Furniture', 'Office Supplies', 'Technology'
len(df["Sub-Category"].unique()) #17 registros
len(list(df["Product Name"].unique())) #1849 productos

1849

In [15]:
#=== Revisando si hay datos de texto en campos numerico (sales) ===

#Itera y comprueba si hay letras en el campo sales(type=int)
for index, row in df.iterrows():
    venta = row['Sales']
    row_id = index
    if re.search(r'[a-zA-Z]', str(venta)): #Busca registros con coincidencias de letras
        print(f"index ={row_id} , value= {venta}")

index =181 , value= ImationÂ Secure+ Hardware Encrypted USB 2.0Â Flash Drive
index =430 , value= ImationÂ Secure+ Hardware Encrypted USB 2.0Â Flash Drive
index =431 , value= ImationÂ Secure+ Hardware Encrypted USB 2.0Â Flash Drive
index =1406 , value= ImationÂ Secure+ Hardware Encrypted USB 2.0Â Flash Drive
index =1969 , value= ImationÂ Secure+ Hardware Encrypted USB 2.0Â Flash Drive
index =1971 , value= ImationÂ Secure+ Hardware Encrypted USB 2.0Â Flash Drive


In [16]:
#=== % de valores nulos===
df.isnull().sum()*100 / len(df)

Row ID           0.0
Order ID         0.0
Order Date       0.0
Ship Date        0.0
Ship Mode        0.0
Customer ID      0.0
Customer Name    0.0
Segment          0.0
Country          0.0
City             0.0
State            0.0
Postal Code      0.0
Region           0.0
Product ID       0.0
Category         0.0
Sub-Category     0.0
Product Name     0.0
Sales            0.0
dtype: float64

In [18]:
#===Guardado de dataframe extracción en archivo pickle ===
df.to_pickle('extracción.pkl')

<p style="font-size:19px; color:blue">📌Nota: </p>


- No se encontrarón registros nulos en los datos

- Los registros no presentan ninguna incosistencia(fallas ortograficas)

- Se detecto que en el indice 181,430,431,1406,1969,1971 en el campo venta hay registros que no concuerdan como venta "ImationÂ Secure+ Hardware Encrypted USB 2.0Â Flash Drive"


- Los datos son todo del tipo object, lo cual hay que corregir porque hay datos que deben ser tipo fecha y numero (Ventas, fecha de entrega, etc).

- Los nombres de columnas se podrian cambiar para mejor entendimiento del campo.